# Loss-Divincenzo qubit control

### Introduction
In this tutorial we will demonstrate coherent control of a Loss-DiVincenzo (LD) single-spin qubit

In [ ]:
import Pyro4
from qick import QickConfig

from spinqick.settings import file_settings
from spinqick.experiments import ld_single_qubit
from spinqick.helper_functions import hardware_manager, file_manager
from spinqick.models import ld_qubit_models, full_experiment_model

In [ ]:
Pyro4.config.SERIALIZER = "pickle"
Pyro4.config.PICKLE_PROTOCOL_VERSION = 4

ns_host = "192.168.2.99"  # make sure this matches your board's ip address!
ns_port = 8888
proxy_name = "myqick"

ns = Pyro4.locateNS(host=ns_host, port=ns_port)
soc = Pyro4.Proxy(ns.lookup(proxy_name))
soccfg = QickConfig(soc.get_cfg())
print(soccfg)

First we will add a qubit definition for this qubit to our config, as we did in the `04_pauli_spin_blockade.ipynb` tutorial notebook.

In [ ]:
full = file_manager.load_config_json(
    file_settings.dot_experiment_config, full_experiment_model.ExperimentConfig
)
ldq_params = ld_qubit_models.Ld1QubitParams(
    gate="P2", f0=5000, pulses=ld_qubit_models.RfPulses(gain_90=0, time_90=1), rf_gen=1
)
full.qubit_configs["q1"].qubit_params = ldq_params

# if you want to save this config and use it
file_manager.save_config_json(full, file_settings.dot_experiment_config)

We will also create a DC source object, and use the `LDSingleQubit` experiment class to initialize the experiment, before giving our qubit an alias `q1`.

In [ ]:
vdc = hardware_manager.DummyDCSource()
ld = ld_single_qubit.LDSingleQubit(soccfg, soc, vdc, plot=False, save_data=False)
ld.qubit = "q1"

### Spectroscopy and coherent control of LD qubits
The following experiments presume the use of Pauli spin blockade initialization and readout, and require SPAM parameters to be determined. To see how this is done, refer back to the PSB tutorial found in the `04_pauli_spin_blockade.ipynb` notebook.

Following optimization in SPAM, we first must determine the frequency of our LD spin-qubit. To do this we will utilize the power of the software defined radio features of the RFSoC, turning one of the generator outputs to a highly-coherent RF source. Because the RFSoC utilizes direct digital synthesis (DDS) to generate the RF drive tones, we can sweep this frequency in "real-time", with the output frequency (or amplitude and phase), changing within a single clock cycle (~2.2 nanoseconds). Because of this speed and the precise timing of the readout sequence, we can run high-resolution frequency sweeps in seconds, removing the need to use chirping to broaden the qubit transition.

To do this, we can simply run the `rf_freq_scan` function once the SPAM config has been properly set:

In [ ]:
ld.rf_freq_scan(rf_gain=0.5, start_freq=5510, stop_freq=5590, rf_length=100, point_avgs=10, full_avgs=10, num_pts=100)

![resonance](demo_data/1740346983_RF_freq_scan.png)

Now that we know where our qubit frequency is located, we can measure a Rabi chevron, showing the coherent oscillation of the qubit state as a function of frequency and time. Centering our scan on the right dip, and fixing the drive power, we can extract the chevron using the `rabi_chevron` method:

In [ ]:
ld.rabi_chevron(rf_gain=0.5, freq_range=(5554, 5557.5, 100), time_range=(0.01, 10, 100), point_avgs=2, full_avgs=2)

![rabichevron](demo_data/1740778116_RabiChevron_scan.png)